In [ ]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
import csv
import time
import ssl

In [ ]:
ssl._create_default_https_context = ssl._create_unverified_context

def merge_intervals(intervals):
    """Объединение перекрывающихся интервалов"""
    if not intervals:
        return []
    sorted_intervals = sorted(intervals, key=lambda x: x[0])
    merged = [sorted_intervals[0]]
    for current in sorted_intervals[1:]:
        last = merged[-1]
        if current[0] <= last[1] + 1:
            merged[-1] = (last[0], max(last[1], current[1]))
        else:
            merged.append(current)
    return merged

def calculate_coverage(intervals, query_length):
    """Расчет процента покрытия"""
    merged = merge_intervals(intervals)
    total = sum(end - start + 1 for start, end in merged)
    return (total / query_length) * 100


def process_blast(sequence):
    """Выполнение BLASTP и парсинг результатов"""
    try:
        result_handle = NCBIWWW.qblast(
            program="blastp",
            database="nr",
            sequence=sequence,
            expect=0.001,
            hitlist_size=1,
            #timeout=300
        )
        blast_record = next(NCBIXML.parse(result_handle))
        result_handle.close()
        
        if not blast_record.alignments:
            return None

        best_hit = blast_record.alignments[0]
        intervals = []
        
        for hsp in best_hit.hsps:
            start = min(hsp.query_start, hsp.query_end)
            end = max(hsp.query_start, hsp.query_end)
            intervals.append((start, end))
        
        coverage = calculate_coverage(intervals, len(sequence))
        best_evalue = min(hsp.expect for hsp in best_hit.hsps)
        main_description = best_hit.hit_def.split('>')[0].strip()

        print(result_handle)

        return {
            'description': main_description,
            'accession': best_hit.accession,
            'coverage': round(coverage, 2),
            'evalue': best_evalue
        }
    
    except Exception as e:
        print(f"Error processing sequence: {str(e)}")
        return None

def main(input_csv, output_csv, sequence_column):
    """Основная функция обработки"""
    # Подсчет общего количества запросов
    with open(input_csv, 'r') as f:
        total = sum(1 for _ in csv.DictReader(f)) - 1  # Исключаем заголовок
    
    # Инициализация прогресса
    processed = 0
    start_time = time.time()
    
    with open(input_csv, 'r') as infile, open(output_csv, 'w', newline='') as outfile:
        reader = csv.DictReader(infile)
        writer = csv.DictWriter(outfile, fieldnames=[
            'Original_Sequence', 'Description', 'Accession', 'Coverage', 'Evalue'
        ])
        writer.writeheader()
        
        for row in reader:
            processed += 1
            seq = row[sequence_column].strip()
            
            # Вывод прогресса
            progress = processed / total * 100
            elapsed = time.time() - start_time
            eta = (elapsed / processed) * (total - processed) if processed else 0
            print(f"\r[{time.strftime('%H:%M:%S')}] "
                  f"Progress: {processed}/{total} ({progress:.1f}%) "
                  f"ETA: {time.strftime('%H:%M:%S', time.gmtime(eta))}",
                  end="", flush=True)
            
            # Обработка последовательности
            result = process_blast(seq)
            time.sleep(5)  # Соблюдение лимитов NCBI
            
            # Запись результатов
            writer.writerow({
                'Original_Sequence': seq,
                'Description': result['description'] if result else 'N/A',
                 'Accession': result['accession'] if result else 'N/A',
                'Coverage': f"{result['coverage']}%" if result else 'N/A',
                'Evalue': f"{result['evalue']:.2e}" if result else 'N/A'
            })
    
    print(f"\n\nTotal time: {time.strftime('%H:%M:%S', time.gmtime(time.time() - start_time))}")

if __name__ == "__main__":
    main(
        input_csv="proteins_1_copy.csv",
        output_csv="results_1_copy.csv",
        sequence_column="content"
    )

[15:00:26] Progress: 1/466 (0.2%) ETA: 00:00:00

c:\Users\anna6\OneDrive\Документы\Blast_for_prot_names\.venv\lib\site-packages\Bio\Blast\NCBIWWW.py:275: BiopythonWarning: BLAST request 0W72BT8W016 is taking longer than 10 minutes, consider re-issuing it
  warnings.warn(


[15:15:32] Progress: 2/466 (0.4%) ETA: 10:25:29<_io.StringIO object at 0x0000025857473790>
[15:17:52] Progress: 3/466 (0.6%) ETA: 20:52:16<_io.StringIO object at 0x0000025857473790>
[15:27:12] Progress: 4/466 (0.9%) ETA: 03:32:41

c:\Users\anna6\OneDrive\Документы\Blast_for_prot_names\.venv\lib\site-packages\Bio\Blast\NCBIWWW.py:275: BiopythonWarning: BLAST request 0W8MHPEV016 is taking longer than 10 minutes, consider re-issuing it
  warnings.warn(


[15:46:32] Progress: 5/466 (1.1%) ETA: 22:51:19<_io.StringIO object at 0x0000025857473790>
[15:47:52] Progress: 6/466 (1.3%) ETA: 12:37:23<_io.StringIO object at 0x0000025857473790>
[15:57:13] Progress: 7/466 (1.5%) ETA: 14:03:51<_io.StringIO object at 0x0000025857473790>
[16:01:32] Progress: 8/466 (1.7%) ETA: 10:18:32<_io.StringIO object at 0x0000025857473790>
[16:03:52] Progress: 9/466 (1.9%) ETA: 05:41:30<_io.StringIO object at 0x0000025857473790>
[16:06:12] Progress: 10/466 (2.1%) ETA: 01:59:23<_io.StringIO object at 0x0000025857473790>
[16:08:32] Progress: 11/466 (2.4%) ETA: 22:57:14<_io.StringIO object at 0x0000025857473790>
[16:10:52] Progress: 12/466 (2.6%) ETA: 20:25:10<_io.StringIO object at 0x0000025857473790>
[16:13:12] Progress: 13/466 (2.8%) ETA: 18:15:58<_io.StringIO object at 0x0000025857473790>
[16:15:32] Progress: 14/466 (3.0%) ETA: 16:24:58<_io.StringIO object at 0x0000025857473790>
[16:17:52] Progress: 15/466 (3.2%) ETA: 14:48:26<_io.StringIO object at 0x00000258574

...

[09:50:34] Progress: 460/466 (98.7%) ETA: 00:33:31Error processing sequence: <urlopen error [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение>
[09:50:49] Progress: 461/466 (98.9%) ETA: 00:27:52

c:\Users\anna6\OneDrive\Документы\Blast_for_prot_names\.venv\lib\site-packages\Bio\Blast\NCBIWWW.py:275: BiopythonWarning: BLAST request 10XPAWRP013 is taking longer than 10 minutes, consider re-issuing it
  warnings.warn(


[10:28:56] Progress: 462/466 (99.1%) ETA: 00:22:35<_io.StringIO object at 0x00000258574B8F70>
[10:33:16] Progress: 463/466 (99.4%) ETA: 00:16:55<_io.StringIO object at 0x00000258574B8F70>
[10:39:36] Progress: 464/466 (99.6%) ETA: 00:11:17

c:\Users\anna6\OneDrive\Документы\Blast_for_prot_names\.venv\lib\site-packages\Bio\Blast\NCBIWWW.py:275: BiopythonWarning: BLAST request 110HTM63016 is taking longer than 10 minutes, consider re-issuing it
  warnings.warn(


[11:07:56] Progress: 465/466 (99.8%) ETA: 00:05:41<_io.StringIO object at 0x00000258574B8F70>
[11:12:16] Progress: 466/466 (100.0%) ETA: 00:00:00<_io.StringIO object at 0x00000258574B8F70>
[11:16:35] Progress: 467/466 (100.2%) ETA: 23:54:18<_io.StringIO object at 0x00000258574B8F70>


Total time: 20:20:29
